# PubPeek 学者文献分析工具 - 简化版本

本Notebook展示如何使用新的 `ReportGenerator` 类来简化代码，消除重复逻辑。

本Notebook演示如何使用PubPeek进行完整的个人文献分析流程，包括：
- BibTeX文件解析
- 论文去重处理
- CCF分级匹配
- 中科院分区匹配
- 结果导出


In [1]:
# 导入必要的模块
import sys
sys.path.append('../')
from core import Config, BibTexParser, PublicationExtractor, RankingMatcher
from utils import DataProcessor, ReportGenerator  # 新增 ReportGenerator
import pandas as pd

# 创建配置
config = Config('../config.json')

# 初始化组件
bib_parser = BibTexParser()

# 创建PublicationExtractor的配置字典
extractor_config = {
    'timeout': config.get('network.timeout', 120),
    'sleep_interval': config.get('network.sleep_interval', 3),
    'proxies': config.get_proxies()
}

publication_extractor = PublicationExtractor(extractor_config)
ranking_matcher = RankingMatcher(
    ccf_file='../data/CCF2022-UTF8.csv',
    cas_file='../data/FQBJCR2025-UTF8.csv'
)
data_processor = DataProcessor()
report_generator = ReportGenerator()  # 新增报告生成器

print("所有组件初始化完成！")


成功加载CCF数据: 644 条记录
成功加载中科院数据: 21772 条记录
所有组件初始化完成！


In [2]:
# 1. 解析BibTeX文件
print("1. 解析BibTeX文件...")
df = bib_parser.parse_file('../users/default_user/raw/references.bib')
print(f"   解析完成: {len(df)} 篇论文")

# 2. 去重处理
print("\n2. 论文去重处理...")
df_deduplicated = data_processor.deduplicate_papers(df, verbose=False)

# 3. 提取出版物信息
print("\n3. 提取出版物信息...")
valid_papers, publication_counts, publication_types = publication_extractor.extract_unique_publication_abbrs(df_deduplicated)

# 使用新的报告生成器打印处理摘要
report_generator.print_processing_summary(
    df, df_deduplicated, publication_counts, publication_types, None
)


1. 解析BibTeX文件...
   解析完成: 179 篇论文

2. 论文去重处理...

3. 提取出版物信息...
   去重完成: 原始 179 篇 → 去重后 168 篇
   去除重复: 11 篇
   找到 64 个出版物: 26 个期刊，38 个会议
   论文分布: 55 篇期刊论文，111 篇会议论文


In [3]:
# 4. 获取DBLP出版物信息
print("\n4. 获取DBLP出版物信息...")
cache_file = '../cache/dblp_cache.json'
publications_info = publication_extractor.batch_extract_publication_info(
    publication_counts, 
    publication_types,
    cache_file=cache_file
)

# 5. 匹配出版物分区
print("\n5. 匹配出版物分区...")
ranking_results = ranking_matcher.batch_match_publications(publications_info)

# 6. 生成统计摘要
print("\n6. 生成统计摘要...")
detailed_summary = ranking_matcher.generate_detailed_summary(ranking_results, publication_counts)

print("✅ 数据处理完成！")



4. 获取DBLP出版物信息...

5. 匹配出版物分区...

6. 生成统计摘要...
✅ 数据处理完成！


In [4]:
# 7. 生成和保存完整报告 - 新的简化方式！
print("\n7. 生成和保存完整报告...")

# 为每篇论文添加CCF和中科院分区信息
df_with_rankings = data_processor.add_ranking_info_to_papers(df_deduplicated, ranking_results)

# 使用新的报告生成器，一次性完成所有保存和统计分析
# 之前这里需要50多行代码，现在只需要一个函数调用！
output_dir = '../users/default_user/processed'
report_results = report_generator.generate_and_save_complete_report(
    df_with_rankings,
    ranking_results,
    publication_counts,
    detailed_summary,
    output_dir,
    data_processor
)

print("✅ 报告生成完成！")



7. 生成和保存完整报告...
为论文添加分区信息...
分区信息添加完成:
  处理论文数: 159
  CCF匹配数: 93
  中科院匹配数: 33
✅ 报告生成完成！


In [5]:
# 8. 打印统计结果 - 也可以选择不同的打印格式
ranking_stats = report_results['ranking_stats']

# 方式1：打印最终统计结果（简洁版）
print("\n=== 方式1：最终统计结果 ===")
report_generator.print_final_statistics(detailed_summary, ranking_stats)

# 方式2：打印详细的论文统计分析（详细版
print("\n=== 方式2：详细论文统计分析 ===")
report_generator.print_detailed_paper_statistics(ranking_stats)

print(f"\n✅ 处理完成！结果已保存到: {output_dir}")
print(f"📁 生成的文件:")
print(f"  - 带分区信息的论文: {report_results['ranked_files']['excel']}")
print(f"  - 详细匹配结果: {report_results['rankings_excel']}")
print(f"  - 统计摘要JSON: {report_results['summary_json']}")

# 显示特殊论文列表
if 'ccf_a_cas_1' in report_results['special_papers_files']:
    print(f"  - CCF-A类+中科院1区论文: {report_results['special_papers_files']['ccf_a_cas_1']['excel']}")
if 'ccf_ab_cas_12' in report_results['special_papers_files']:
    print(f"  - CCF-A/B类+中科院1/2区论文: {report_results['special_papers_files']['ccf_ab_cas_12']['excel']}")



=== 方式1：最终统计结果 ===

=== 分区匹配结果统计 ===
CCF分级统计 (论文数量):
  总匹配: 40/64 个出版物
  期刊论文: A类=5篇, B类=13篇, C类=5篇
  会议论文: A类=27篇, B类=30篇, C类=14篇

中科院分区统计 (期刊论文数量):
  总匹配: 22 个期刊
  期刊论文: 1区=11篇, 2区=8篇, 3区=9篇, 4区=5篇
  Top期刊论文: 11篇

=== 论文分区统计总结 ===
📊 总论文数: 168
🏆 CCF-A类+中科院1区: 37篇
🌟 CCF-A/B类+中科院1/2区: 85篇
🎯 CCF-A类: 31篇 | CCF-B类: 43篇 | CCF-C类: 19篇
⭐ 中科院TOP: 11篇
🔬 中科院1区: 11篇 | 2区: 8篇 | 3区: 9篇 | 4区: 5篇

=== 方式2：详细论文统计分析 ===
=== 论文分区统计分析 ===

📊 总论文数: 168

=== CCF分区统计 ===
🅰️  CCF-A类论文: 31篇 (18.5%)
🅱️  CCF-B类论文: 43篇 (25.6%)
🅲  CCF-C类论文: 19篇 (11.3%)

=== 中科院分区统计 ===
🥇 中科院1区论文: 11篇 (6.5%)
⭐ 中科院1区TOP论文: 11篇 (6.5%)
🥈 中科院2区论文: 8篇 (4.8%)
🥉 中科院3区论文: 9篇 (5.4%)
🏅 中科院4区论文: 5篇 (3.0%)

=== 组合统计 (去重) ===
🏆 CCF-A类 + 中科院1区 (并集): 37篇 (22.0%)
🌟 CCF-A/B类 + 中科院1/2区 (并集): 85篇 (50.6%)

✅ 处理完成！结果已保存到: ../users/default_user/processed
📁 生成的文件:
  - 带分区信息的论文: ../users/default_user/processed/papers_with_rankings.xlsx
  - 详细匹配结果: ../users/default_user/processed/journal_rankings.xlsx
  - 统计摘要JSON: ../users/default_user/processed/summary

In [1]:
# 年份区间